In [1]:
import json
import openai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = openai.Client()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def saudacao_por_periodo(hora):
    if 5 <= hora < 12:
        return json.dumps({"saudacao": "Bom dia!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao": "Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "Boa noite!"})
    else:
        return json.dumps({"saudacao": "Boa madrugada!"})

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudação baseada na hora do dia",
            "parameters": {
                "type": "object",
                "properties": {
                    "hora": {
                        "type": "integer",
                        "description": "A hora do dia em formato de 24h"
                    }
                },  
                "required": ["hora"]
            }
        }
    }
]

In [10]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [11]:
mensagens = [{"role": "user", "content": "Qual saudação o modelo me dá e for 10h?"}]

In [12]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice="auto"
)

In [13]:
menssagem_resp = resposta.choices[0].message
menssagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dn7HqjgE6M4o6rmX2xL5wQ8M', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')])

In [14]:
tool_calls = menssagem_resp.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_dn7HqjgE6M4o6rmX2xL5wQ8M', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')]

In [ ]:
if tool_calls:
    mensagens.append(menssagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )

        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )
    
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model="gpt-3.5-turbo-0125"
    )